In [1]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
provider = SaverProvider() 

In [4]:
from Scratch import metadata_loader

df = metadata_loader(True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]

df = df[df['backend_name']=='ibm_sherbrooke']
df = df[df['rounds']==100]
df = df[df['xbasis'] == True]
df = df[df['logical'] == "0"]

df

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets,num_qubit,double_msmt,xbasis,path_info
5315,2024-03-23 17:56:21.062000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgkd5ktf3g00883v1g,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5314,2024-03-23 17:56:05.176000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgk95dvs8g008j6ep0,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5313,2024-03-23 17:55:48.840000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgk54czq6g0081hjy0,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5312,2024-03-23 17:55:33.296000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgk1cktf3g00883v00,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5311,2024-03-23 17:55:17.834000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgjxbk5z70008j0de0,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5310,2024-03-23 17:55:02.330000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgjskdvs8g008j6emg,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5309,2024-03-23 17:54:47.141000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgjnj8gdp0008fvyt0,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5308,2024-03-23 17:54:31.495000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgjhtktf3g00883tz0,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5307,2024-03-23 17:54:15.732000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgjdsczq6g0081hjtg,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5306,2024-03-23 17:54:01.026000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgja1s9z7g008dpg5g,NaN,{},"[Subset 52, 100 rounds, 0 log, xbasis=True]",1.0,True,single,...,0,NaN,subset RepCodes,NaN,NaN,NaN,NaN,NaN,True,"{'mean_gate_error': 0.008345314216382, 'min_ga..."


In [5]:
from time import sleep

OVERWRITE = False
IGNORE_RUNNING = False

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=IGNORE_RUNNING, overwrite=OVERWRITE)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            # break  
            sleep(5)


  0%|          | 0/20 [00:00<?, ?it/s]

19:37:28 Warning: Job ID cqzgkd5ktf3g00883v1g not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
19:38:30 Warning: Job ID cqzgk95dvs8g008j6ep0 not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
19:39:32 Warning: Job ID cqzgk54czq6g0081hjy0 not found in /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft

# CLEANUP: Check if jobs have files

In [5]:
md = metadata_loader(True)

for job_id in tqdm(md.job_id):

    filename = provider.job_local_filename(job_id)
    status = md.loc[md['job_id'] == job_id, 'job_status'].values[0]

    if filename == None and status not in ['JobStatus.ERROR', 'JobStatus.CANCELLED']:
        provider.retrieve_job(job_id)

  0%|          | 0/4223 [00:00<?, ?it/s]

# update the metadata with the execution date

In [ ]:
from Scratch import update_metadata

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.ERROR', 'JobStatus.CANCELLED'])]

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            ibm_job = provider.retrieve_job(job_id, _ignore_running=False)
            additional_dict = {"execution_date": str(ibm_job.result().date)}
            update_metadata(ibm_job, "None_Backend", additional_dict)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  
            